In [1]:
DATA_NAME = 'syntheticMRI2D-axial' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_diagonal_wavelet_axial_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,3,4,5,6,7,8,9
3,2839499.07,-24.66,5.65,-3.72,-0.44,0.14,0.05
4,-24.66,819110.29,-6.49,-0.91,0.68,0.05,-0.01
5,5.65,-6.49,152401.57,-2.13,0.28,0.17,0.00
6,-3.72,-0.91,-2.13,55942.41,0.11,-0.05,0.00
7,-0.44,0.68,0.28,0.11,3509.95,0.00,0.00
8,0.14,0.05,0.17,-0.05,0.00,100.76,-0.00
9,0.05,-0.01,0.00,0.00,0.00,-0.00,2.15


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

,3,4,5,6,7,8,9
3,1.00000,-0.00002,0.00001,-0.00001,-0.00000,0.00001,0.00002
4,-0.00002,1.00000,-0.00002,-0.00000,0.00001,0.00001,-0.00001
5,0.00001,-0.00002,1.00000,-0.00002,0.00001,0.00004,0.00000
6,-0.00001,-0.00000,-0.00002,1.00000,0.00001,-0.00002,0.00001
7,-0.00000,0.00001,0.00001,0.00001,1.00000,0.00000,0.00003
8,0.00001,0.00001,0.00004,-0.00002,0.00000,1.00000,-0.00001
9,0.00002,-0.00001,0.00000,0.00001,0.00003,-0.00001,1.00000


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

37.47694955585719

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.15482390e+12 9.31070739e+10 3.12233405e+09 3.87512535e+08
 1.36845949e+06 8.46244826e+02 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,9.985925e-01,-0.052204,-0.008797,-0.003222,-0.000202,-0.000006,-1.077060e-07
1,5.177487e-02,0.997804,-0.038948,-0.013777,-0.000859,-0.000025,-5.400508e-07
2,1.033787e-02,0.036797,0.993718,-0.105036,-0.005490,-0.000156,-3.367097e-06
3,5.037387e-03,0.017516,0.104260,0.994146,-0.021694,-0.000620,-1.318881e-05
4,4.123138e-04,0.001428,0.007680,0.020972,0.999646,-0.014378,-3.062660e-04
5,1.768503e-05,0.000061,0.000329,0.000901,0.014350,0.999668,-2.138283e-02
6,7.413644e-07,0.000003,0.000014,0.000038,0.000613,0.021376,9.997713e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0014075176031888015,
 0.002196463716329422,
 0.0062815080705427295,
 0.005853804292249576,
 0.00035398607898851076,
 0.0003320950659240207,
 0.00022868588447622074]